## Day 7

### Part 1
You land at the regional airport in time for your next flight. In fact, it looks like you'll even have time to grab some food: all flights are currently delayed due to issues in luggage processing.

Due to recent aviation regulations, many rules (your puzzle input) are being enforced about bags and their contents; bags must be color-coded and must contain specific quantities of other color-coded bags. Apparently, nobody responsible for these regulations considered how long they would take to enforce!

For example, consider the following rules:

- light red bags contain 1 bright white bag, 2 muted yellow bags.
- dark orange bags contain 3 bright white bags, 4 muted yellow bags.
- bright white bags contain 1 shiny gold bag.
- muted yellow bags contain 2 shiny gold bags, 9 faded blue bags.
- shiny gold bags contain 1 dark olive bag, 2 vibrant plum bags.
- dark olive bags contain 3 faded blue bags, 4 dotted black bags.
- vibrant plum bags contain 5 faded blue bags, 6 dotted black bags.
- faded blue bags contain no other bags.
- dotted black bags contain no other bags.

These rules specify the required contents for 9 bag types. In this example, every faded blue bag is empty, every vibrant plum bag contains 11 bags (5 faded blue and 6 dotted black), and so on.

You have a shiny gold bag. If you wanted to carry it in at least one other bag, how many different bag colors would be valid for the outermost bag? (In other words: how many colors can, eventually, contain at least one shiny gold bag?)

In the above rules, the following options would be available to you:

- A bright white bag, which can hold your shiny gold bag directly.
- A muted yellow bag, which can hold your shiny gold bag directly, plus some other bags.
- A dark orange bag, which can hold bright white and muted yellow bags, either of which could then hold your shiny gold bag.
- A light red bag, which can hold bright white and muted yellow bags, either of which could then hold your shiny gold bag.

So, in this example, the number of bag colors that can eventually contain at least one shiny gold bag is 4.

How many bag colors can eventually contain at least one shiny gold bag? (The list of rules is quite long; make sure you get all of it.)

In [1]:
# import packages
import pandas as pd
import numpy as np

In [2]:
# read in data
rules = pd.read_csv('input_data/Day7.txt', sep=":", header=None)

In [3]:
rules.shape

(594, 1)

In [4]:
rules.columns=['rule']

In [5]:
rules.head()

,rule
0,"muted lime bags contain 1 wavy lime bag, 1 vib..."
1,"light red bags contain 2 clear indigo bags, 3 ..."
2,wavy beige bags contain 4 faded chartreuse bags.
3,muted blue bags contain 3 mirrored tan bags.
4,"vibrant cyan bags contain 4 drab beige bags, 4..."


In [6]:
# Let's check consistency of data
np.sum(rules['rule'].str.find(' bags contain ')>0)

594

In [7]:
# Define a function to break out the container bag
def container_bag(rule):
    '''returns container bag for a given rule based on everythign to the left of the word "contain"'''
    ind = rule.find(' bags contain ')
    return rule[0:ind]

In [8]:
container_bag(rules['rule'][0])

'muted lime'

In [9]:
rules['outside_bag'] = rules.apply(lambda x: container_bag(x['rule']), axis=1)

In [10]:
rules['outside_bag']

0           muted lime
1            light red
2           wavy beige
3           muted blue
4         vibrant cyan
            ...       
589     dull turquoise
590        bright aqua
591        light olive
592    pale chartreuse
593        dim magenta
Name: outside_bag, Length: 594, dtype: object

In [11]:
# Define a function to build a list of contained bags
def inside_bags(rule):
    '''returns a list of dictionaries of inside bags with key value pairs of number and color
    for a given rule based on everything to the right of the word "contain"'''
    
    # Let's clean up the string a bit
    # First get rid of everything on the left inc. the word
    ind = rule.find(' bags contain ')+ len(' bags contain ')
    # Now get rid of bag, bags, and periods
    ins_str = rule[ind:]
    ins_str = ins_str.replace('.','')
    ins_str = ins_str.replace(' bags','')
    ins_str = ins_str.replace(' bag','')
    ins_str = ins_str.strip()
   
    # Let's initialize our dictionary
    ins_bags=[]
    
    # if the rest is equal to "no other" return null
    if ins_str=='no other':
        return ''
    else:# Now break up the rest into a list of contained bags
        inside = ins_str.split(',')
        inside = [bag.strip() for bag in inside]
        return [{"number": bag[0:bag.find(' ')], "color" : bag[bag.find(' ')+1:]} for bag in inside]

In [12]:
# Let's test this
inside_bags(rules['rule'][0])

[{'number': '1', 'color': 'wavy lime'},
 {'number': '1', 'color': 'vibrant green'},
 {'number': '3', 'color': 'light yellow'}]

In [13]:
# Now let's add a column to our df containing this list
rules['inside_bags'] = rules.apply(lambda x: inside_bags(x['rule']), axis=1)

In [14]:
rules

,rule,outside_bag,inside_bags
0,"muted lime bags contain 1 wavy lime bag, 1 vib...",muted lime,"[{'number': '1', 'color': 'wavy lime'}, {'numb..."
1,"light red bags contain 2 clear indigo bags, 3 ...",light red,"[{'number': '2', 'color': 'clear indigo'}, {'n..."
2,wavy beige bags contain 4 faded chartreuse bags.,wavy beige,"[{'number': '4', 'color': 'faded chartreuse'}]"
3,muted blue bags contain 3 mirrored tan bags.,muted blue,"[{'number': '3', 'color': 'mirrored tan'}]"
4,"vibrant cyan bags contain 4 drab beige bags, 4...",vibrant cyan,"[{'number': '4', 'color': 'drab beige'}, {'num..."
...,...,...,...
589,"dull turquoise bags contain 5 light teal bags,...",dull turquoise,"[{'number': '5', 'color': 'light teal'}, {'num..."
590,bright aqua bags contain 4 dotted lime bags.,bright aqua,"[{'number': '4', 'color': 'dotted lime'}]"
591,"light olive bags contain 3 wavy lavender bags,...",light olive,"[{'number': '3', 'color': 'wavy lavender'}, {'..."
592,"pale chartreuse bags contain 4 dark lime bags,...",pale chartreuse,"[{'number': '4', 'color': 'dark lime'}, {'numb..."


In [15]:
# Now define a function to return whether a color is in the list of dictionaries in the column inside_bags
def find_color(bag_list, color):
    '''returns true if color is in the bag_list which is a list of dictionaries'''
    return any(bag['color'] == color for bag in bag_list)

In [16]:
# Let's test this
find_color(rules['inside_bags'][0],'wavy lime')

True

In [17]:
# Here is our list of outside bags that can contain shiny gold
rules[rules.apply(lambda x: find_color(x['inside_bags'],'shiny gold'), axis=1)]['outside_bag'].to_list()

['dark black', 'dim beige', 'mirrored red', 'posh brown', 'shiny gray']

In [18]:
# Define a function to return a list of bags for a particular color
def list_of_bags(color):
    '''returns a list of container bags for a particular color'''
    return rules[rules.apply(lambda x: find_color(x['inside_bags'], color), axis=1)]['outside_bag'].to_list()

In [19]:
# Let's get a unique list of colors based on an initial list
def get_new_list(initial_list):
    '''takes an initial list of colors and returns a new list of colors'''
    new_list=[]
    
    # Create a list of possible new colors
    for color in initial_list:
        for i in list_of_bags(color):
            if i not in new_list:
                new_list.append(i)

    return new_list

In [20]:
total_list = ['shiny gold']
keep_adding_colors = True
depth = 0

while keep_adding_colors:
    depth+=1
    print(depth, len(total_list))
    
    # we get our new list of colors and see if there's anything new in there
    keep_adding_colors = False

    for i in get_new_list(total_list):
        if i not in total_list:
            keep_adding_colors=True
            total_list.append(i)

1 1
2 6
3 22
4 46
5 70
6 96
7 125
8 139
9 140


Answer is actually 139 because we don't count the shiny gold bag!

### Part 2
It's getting pretty expensive to fly these days - not because of ticket prices, but because of the ridiculous number of bags you need to buy!

Consider again your shiny gold bag and the rules from the above example:

- faded blue bags contain 0 other bags.
- dotted black bags contain 0 other bags.
- vibrant plum bags contain 11 other bags: 5 faded blue bags and 6 dotted black bags.
- dark olive bags contain 7 other bags: 3 faded blue bags and 4 dotted black bags.

So, a single shiny gold bag must contain 1 dark olive bag (and the 7 bags within it) plus 2 vibrant plum bags (and the 11 bags within each of those): 1 + 1*7 + 2 + 2*11 = 32 bags!

Of course, the actual rules have a small chance of going several levels deeper than this example; be sure to count all of the bags, even if the nesting becomes topologically impractical!

Here's another example:

- shiny gold bags contain 2 dark red bags.
- dark red bags contain 2 dark orange bags.
- dark orange bags contain 2 dark yellow bags.
- dark yellow bags contain 2 dark green bags.
- dark green bags contain 2 dark blue bags.
- dark blue bags contain 2 dark violet bags.
- dark violet bags contain no other bags.

In this example, a single shiny gold bag must contain 126 other bags.

How many individual bags are required inside your single shiny gold bag?



In [21]:
def get_list_of_colors(color):
    '''this function returns the list of dictionaries of inside bags for a given outside bag'''
    return rules[rules['outside_bag']==color]['inside_bags'].to_list()[0]

In [25]:
# Define our recursive function

def num_bags_inside(color, number):
    
    # we always return the number of bags for this level
    num_bags = number

    # then we add the number of bags inside that bag
    new_list = get_list_of_colors(color)
    if new_list != "":
        for item in new_list:
            num_bags += number * num_bags_inside(item.get('color'), int(item.get('number')))
    return num_bags

In [26]:
num_bags_inside('shiny gold', 1)

58176

And actual answer is 1 less or 58175 because we don't count the shiny gold bag.